<a href="https://www.kaggle.com/code/kagglebenb/penguin-machine-learning?scriptVersionId=107235153" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages
library(ggplot2)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


[1] "palmer-archipelago-antarctica-penguin-data"

This is a follow up to the first penguin notebook in R. This notebook is also in R, but will be focused on Machine Learning.

Some goals for this notebook
 * Based on culmen length/culmen depth/flipper length/body mass, can we determine 
      - species?
      - island?
 * Given island/species/culmen length/culmen depth/flipper length/body mass, can we determine
      - sex?
 * Given island/species/stage/date egg, can we determine
      - clutch completion?
 * Given culmen length, can we determine
      - culmen depth?
      - flipper length?
      - body mass?
      - what if species was also a feature?
 * Given flipper length, can we determine
      - body mass?
      - what if species was also a feature?
 * Given species/island/clutch completion/date egg, can we determine
      - stage?
          - this one is probably not going to go well

In [2]:
penguins_size <- read.csv('../input/palmer-archipelago-antarctica-penguin-data/penguins_size.csv', header=TRUE)
head(penguins_size)

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
,<chr>,<chr>,<dbl>,<dbl>,<int>,<int>,<chr>
1,Adelie,Torgersen,39.1,18.7,181,3750,MALE
2,Adelie,Torgersen,39.5,17.4,186,3800,FEMALE
3,Adelie,Torgersen,40.3,18.0,195,3250,FEMALE
4,Adelie,Torgersen,NA,NA,NA,NA,NA
5,Adelie,Torgersen,36.7,19.3,193,3450,FEMALE
6,Adelie,Torgersen,39.3,20.6,190,3650,MALE


In [3]:
penguin_lengths <- penguins_size[c("species","island","culmen_length_mm", "culmen_depth_mm", "flipper_length_mm")]
head(penguin_lengths)

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm
,<chr>,<chr>,<dbl>,<dbl>,<int>
1,Adelie,Torgersen,39.1,18.7,181
2,Adelie,Torgersen,39.5,17.4,186
3,Adelie,Torgersen,40.3,18.0,195
4,Adelie,Torgersen,NA,NA,NA
5,Adelie,Torgersen,36.7,19.3,193
6,Adelie,Torgersen,39.3,20.6,190


Let's drop all NA values.

In [4]:
dataset <- penguin_lengths[complete.cases(penguin_lengths), ]
head(dataset)

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm
,<chr>,<chr>,<dbl>,<dbl>,<int>
1,Adelie,Torgersen,39.1,18.7,181
2,Adelie,Torgersen,39.5,17.4,186
3,Adelie,Torgersen,40.3,18.0,195
5,Adelie,Torgersen,36.7,19.3,193
6,Adelie,Torgersen,39.3,20.6,190
7,Adelie,Torgersen,38.9,17.8,181


In [5]:
# Machine Learning Library
library(caret)

Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift


The following object is masked from ‘package:httr’:

    progress




In [6]:
dummy <- dummyVars(" ~ .", data=dataset)
dataset_with_dummies <- data.frame(predict(dummy, newdata=dataset))
head(dataset_with_dummies)

,speciesAdelie,speciesChinstrap,speciesGentoo,islandBiscoe,islandDream,islandTorgersen,culmen_length_mm,culmen_depth_mm,flipper_length_mm
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,0,0,0,0,1,39.1,18.7,181
2,1,0,0,0,0,1,39.5,17.4,186
3,1,0,0,0,0,1,40.3,18.0,195
5,1,0,0,0,0,1,36.7,19.3,193
6,1,0,0,0,0,1,39.3,20.6,190
7,1,0,0,0,0,1,38.9,17.8,181


In [7]:
dataset_with_dummies$id <- 1:nrow(dataset_with_dummies)

#use 70% of dataset as training set and 30% as test set 
train <- dataset_with_dummies %>% dplyr::sample_frac(0.70)
test  <- dplyr::anti_join(dataset_with_dummies, train, by = 'id')

In [8]:
head(train)

,speciesAdelie,speciesChinstrap,speciesGentoo,islandBiscoe,islandDream,islandTorgersen,culmen_length_mm,culmen_depth_mm,flipper_length_mm,id
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
235,0,0,1,1,0,0,45.8,14.6,210,234
171,0,1,0,0,1,0,46.4,18.6,190,170
159,0,1,0,0,1,0,46.1,18.2,178,158
110,1,0,0,1,0,0,43.2,19.0,197,109
262,0,0,1,1,0,0,49.6,16.0,225,261
321,0,0,1,1,0,0,48.5,15.0,219,320


In [9]:
head(test)

,speciesAdelie,speciesChinstrap,speciesGentoo,islandBiscoe,islandDream,islandTorgersen,culmen_length_mm,culmen_depth_mm,flipper_length_mm,id
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
5,1,0,0,0,0,1,36.7,19.3,193,4
9,1,0,0,0,0,1,34.1,18.1,193,8
10,1,0,0,0,0,1,42.0,20.2,190,9
11,1,0,0,0,0,1,37.8,17.1,186,10
12,1,0,0,0,0,1,37.8,17.3,180,11
19,1,0,0,0,0,1,34.4,18.4,184,18


In [10]:
lr_model <- lm(flipper_length_mm~speciesAdelie+speciesChinstrap+speciesGentoo+islandBiscoe+islandDream+islandTorgersen+culmen_length_mm+culmen_depth_mm, data=train)
summary(lr_model) #Review the results


Call:
lm(formula = flipper_length_mm ~ speciesAdelie + speciesChinstrap + 
    speciesGentoo + islandBiscoe + islandDream + islandTorgersen + 
    culmen_length_mm + culmen_depth_mm, data = train)

Residuals:
     Min       1Q   Median       3Q      Max 
-20.1124  -3.0544   0.0818   3.3901  15.7357 

Coefficients: (2 not defined because of singularities)
                 Estimate Std. Error t value Pr(>|t|)    
(Intercept)      156.4350     6.5149  24.012  < 2e-16 ***
speciesAdelie    -29.4564     2.4430 -12.058  < 2e-16 ***
speciesChinstrap -31.6124     2.0657 -15.304  < 2e-16 ***
speciesGentoo          NA         NA      NA       NA    
islandBiscoe      -2.5801     1.3239  -1.949   0.0525 .  
islandDream       -0.5917     1.3354  -0.443   0.6581    
islandTorgersen        NA         NA      NA       NA    
culmen_length_mm   0.7188     0.1454   4.945 1.46e-06 ***
culmen_depth_mm    1.9771     0.3721   5.313 2.53e-07 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘

In [11]:
predictions <- predict(lr_model, test[c('speciesAdelie', 'speciesChinstrap', 'speciesGentoo', 'islandBiscoe', 'islandDream', 'islandTorgersen' ,"culmen_length_mm", "culmen_depth_mm")])
predictions

Warning message in predict.lm(lr_model, test[c("speciesAdelie", "speciesChinstrap", :
“prediction from a rank-deficient fit may be misleading”


5        9       10       11       12       19       20       31 
191.5154 187.2740 197.1043 187.9565 188.3519 188.0828 202.5497 187.7960 
      32       33       34       35       36       37       39       44 
188.9106 189.9707 193.1518 186.1608 196.2794 193.8171 191.5706 197.0336 
      45       47       48       52       56       57       59       64 
186.3944 193.4933 190.7079 190.5883 190.9296 187.0298 183.4534 189.9232 
      68       71       72       73       76       82       83       88 
191.7025 188.6221 191.8924 189.4481 194.3184 192.6109 190.5268 189.6835 
      91       92       99      101      108      116      126      131 
187.6347 191.7139 182.0095 184.9454 191.3974 191.2710 193.7255 190.0413 
     133      142      145      147      150      152      153      155 
189.4139 189.5752 186.4123 191.3367 189.3419 192.7923 193.0440 199.0644 
     158      161      162      166      179      184      185      189 
191.9119 194.6617 200.4483 197.3928 196.9077 204.3122 187.7964 194.6255 
     194      195      196      199      200      206      207      208 
192.0376 198.5792 190.5459 195.6317 198.2020 199.6217 188.9826 198.9205 
     210      214      218      229      238      239      240      242 
199.0108 190.4918 195.8654 211.4711 219.2707 215.7304 218.7136 214.9397 
     249      251      253      254      255      258      259      260 
216.7725 218.1027 214.9397 230.3048 218.4080 219.9723 212.3697 219.8998 
     261      263      268      269      272      281      282      285 
211.6329 213.5018 221.5890 212.4235 216.5928 213.6995 216.5212 214.8497 
     289      297      298      299      302      304      310      315 
213.1965 216.0717 222.8111 213.6276 222.4335 221.2296 224.9139 214.9038 
     320      328      329      330      341      343      344 
223.2065 223.4758 212.6573 218.2823 215.7662 215.6047 221.5532

In [12]:
test$flipper_length_mm

[1] 193 193 190 186 180 184 194 178 178 188 184 195 196 190 181 196 185 182
 [19] 179 188 191 186 181 192 188 190 190 196 195 196 187 189 202 205 178 192
 [37] 190 196 199 190 193 187 192 190 193 201 192 193 198 195 198 201 200 201
 [55] 187 195 187 196 196 190 212 203 187 197 203 189 193 209 221 209 222 215
 [73] 210 222 207 230 220 219 208 208 208 210 225 213 220 208 221 219 220 209
 [91] 220 215 221 224 230 214 225 219 208 209 215 212 213

In [13]:
predictions <- as.vector(predictions)
predictions

[1] 191.5154 187.2740 197.1043 187.9565 188.3519 188.0828 202.5497 187.7960
  [9] 188.9106 189.9707 193.1518 186.1608 196.2794 193.8171 191.5706 197.0336
 [17] 186.3944 193.4933 190.7079 190.5883 190.9296 187.0298 183.4534 189.9232
 [25] 191.7025 188.6221 191.8924 189.4481 194.3184 192.6109 190.5268 189.6835
 [33] 187.6347 191.7139 182.0095 184.9454 191.3974 191.2710 193.7255 190.0413
 [41] 189.4139 189.5752 186.4123 191.3367 189.3419 192.7923 193.0440 199.0644
 [49] 191.9119 194.6617 200.4483 197.3928 196.9077 204.3122 187.7964 194.6255
 [57] 192.0376 198.5792 190.5459 195.6317 198.2020 199.6217 188.9826 198.9205
 [65] 199.0108 190.4918 195.8654 211.4711 219.2707 215.7304 218.7136 214.9397
 [73] 216.7725 218.1027 214.9397 230.3048 218.4080 219.9723 212.3697 219.8998
 [81] 211.6329 213.5018 221.5890 212.4235 216.5928 213.6995 216.5212 214.8497
 [89] 213.1965 216.0717 222.8111 213.6276 222.4335 221.2296 224.9139 214.9038
 [97] 223.2065 223.4758 212.6573 218.2823 215.7662 215.6047 221.5532

In [14]:
abs_errors <- abs(predictions - test$flipper_length_mm)
abs_errors

[1]  1.48462496  5.72595229  7.10432550  1.95652442  8.35193555  4.08280216
  [7]  8.54966793  9.79597582 10.91063090  1.97073702  9.15180775  8.83916432
 [13]  0.27938286  3.81710465 10.57061467  1.03358834  1.39440563 11.49327182
 [19] 11.70791317  2.58832433  0.07036208  1.02977466  2.45344327  2.07682209
 [25]  3.70252798  1.37787773  1.89240255  6.55194347  0.68163506  3.38910588
 [31]  3.52684723  0.68352096 14.36526345 13.28607825  4.00947027  7.05457319
 [37]  1.39737973  4.72904849  5.27445080  0.04132370  3.58606385  2.57517541
 [43]  5.58766217  1.33674377  3.65809358  8.20773899  1.04402761  6.06440406
 [49]  6.08810824  0.33831302  2.44834300  3.60720237  3.09227447  3.31219140
 [55]  0.79639075  0.37447836  5.03756760  2.57918149  5.45411499  5.63168070
 [61] 13.79799649  3.37834364  1.98262413  1.92049508  3.98924205  1.49178748
 [67]  2.86540113  2.47109282  1.72933114  6.73035234  3.28643297  0.06031944
 [73]  6.77252626  3.89733137  7.93968056  0.30481127  1.59203265  0.97228158
 [79]  4.36965871 11.89980042  3.63293399  3.50179456  3.41096191  0.57654471
 [85]  3.40724711  5.69950012  4.47882541  4.15028136  6.80350417  7.07166593
 [91]  2.81113586  1.37237913  1.43350645  2.77035817  5.08613267  0.90381617
 [97]  1.79345301  4.47583085  4.65732620  9.28229152  0.76621673  3.60467651
[103]  8.55317370

In [15]:
max(abs_errors)

[1] 14.36526

In [16]:
mean(abs_errors)

[1] 4.280752

In [17]:
mse = sum((predictions - test$flipper_length_mm) ^2) / length(predictions)
mse

[1] 29.55983